Implementation of http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html?highlight=mnist

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as Utils

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from tqdm import tnrange
import scipy.ndimage
import cv2
import os
import csv
import pandas as pd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
import imageio


RESIZE = 128
BATCH_SIZE = 256

#### Pokemon Dataloader

In [ ]:
pokelist = os.listdir("./Pokemon/")
with open("./PokeList", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(pokelist)

In [ ]:
# Dataloader class that works with VCTK datasets. Init require the path to a folder where voice_data are stored.
class PokeSet(Utils.Dataset): # Main dataset class for dataloader
    def __init__(self, path): # takes two input, the input and output Tensors
        self.path = path
        self.pokelist = pd.read_csv("PokeList",header=None)

    
    def __len__(self): # must be written for dataset module
        return len(os.listdir(self.path))
    
    def __getitem__(self,index): # must be written for dataset module
        pokemon = mpimg.imread(self.path + self.pokelist[index][0])
        pokemon = cv2.resize(pokemon, (RESIZE,RESIZE))
        pokemon = (pokemon - np.min(pokemon))/np.max(pokemon - np.min(pokemon))
        return pokemon
    
pokeSet = PokeSet("./Pokemon/")
pokeLoader = Utils.DataLoader(dataset = pokeSet, shuffle = True, batch_size = BATCH_SIZE)

In [ ]:
for i, data in enumerate(pokeLoader):
    if i in range(10):
        plt.imshow(data[0,:,:,:].squeeze().numpy())
        plt.show()

#### Define Network

In [ ]:
class Net(nn.Module):
    def __init__(self, latent_dim=256):
        super(Net, self).__init__()
        
        #encoder
        self.conv_e1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        self.bn_e1 = nn.BatchNorm2d(32)
        self.conv_e2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn_e2 = nn.BatchNorm2d(64)
        self.conv_e3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn_e3 = nn.BatchNorm2d(128)
        self.conv_e4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn_e4 = nn.BatchNorm2d(256)
        
        self.fc_mean = nn.Linear(256*RESIZE//16*RESIZE//16, latent_dim)
        self.fc_logvar = nn.Linear(256*RESIZE//16*RESIZE//16, latent_dim)
        
        #decoder
        self.fc_d = nn.Linear(latent_dim, 256*RESIZE//16*RESIZE//16)
        
        self.up = nn.Upsample(scale_factor=2)
        
        self.conv_d1 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.bn_d1 = nn.BatchNorm2d(128)
        self.conv_d2 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.bn_d2 = nn.BatchNorm2d(64)
        self.conv_d3 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.bn_d3 = nn.BatchNorm2d(32)
        self.conv_d4 = nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1)
        
    def encoder(self, x):
        ''' encoder: q(z|x)
            input: x, output: mean, logvar
        '''
        x = F.leaky_relu(self.bn_e1(self.conv_e1(x)))
        x = F.leaky_relu(self.bn_e2(self.conv_e2(x)))
        x = F.leaky_relu(self.bn_e3(self.conv_e3(x)))
        x = F.leaky_relu(self.bn_e4(self.conv_e4(x)))
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        z_mean = self.fc_mean(x)
        z_logvar = self.fc_logvar(x)
        return z_mean, z_logvar
    
    def latent(self, z_mu, z_logvar):
        ''' 
            encoder: z = mu + sd * e
            input: mean, logvar. output: z
        '''
        sd = torch.exp(z_logvar * 0.5)
        e = Variable(torch.randn(sd.size())).cuda()
        z = e.mul(sd).add_(z_mu)
        return z 
    
    def decoder(self, z):
        '''
            decoder: p(x|z)
            input: z. output: x
        '''
        x = self.fc_d(z)
        x = x.view(-1, 256, RESIZE//16, RESIZE//16)
        x = F.leaky_relu(self.bn_d1(self.conv_d1(self.up(x))))
        x = F.leaky_relu(self.bn_d2(self.conv_d2(self.up(x))))
        x = F.leaky_relu(self.bn_d3(self.conv_d3(self.up(x))))
        x = F.sigmoid(self.conv_d4(self.up(x)))
        return x.view(-1,3,RESIZE,RESIZE)

    def forward(self, x):
        z_mean, z_logvar = self.encoder(x)
        z = self.latent(z_mean, z_logvar)
        x_out = self.decoder(z)
        return x_out, z_mean, z_logvar


net = Net()

#### Define Loss Function

Loss Function

$$ 2+2 = 4 - 1 = 3 , Quikmafz$$

## $$l_i(\theta, \phi) = E_{z\sim q_\theta(z\vert x_i)}[\log p_\phi(x_i\vert z)] - KL(q_\theta(z\vert x_i) \vert\vert p(z))$$

In [ ]:
import torch.optim as optim
#Define Criterion

def criterion(x_out, target, z_mean, z_logvar, alpha = 1, beta = 0.5):
    bce = F.mse_loss(x_out, target, size_average=False) #Use MSE loss for images
    kl = -0.5 * torch.sum(1 + z_logvar - (z_mean**2) - torch.exp(z_logvar)) #Analytical KL Divergence - Assumes p(z) is Gaussian Distribution
    loss = ((alpha * bce) + (beta * kl)) / x_out.size(0)    
    return loss, bce, kl

# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 0.001)

#### Train Network

In [ ]:
def load_checkpoint(filename):
    '''Loading function for the model before and during training
    From a checkpoint file, it loads and returns all necessary data (mode, optimiser, epoch number, losses)
    input: filename -> The name of the checkpoint file to be opened (.pth or .pt)
    output: net -> The saved model, including weights and biases
    output: epoch -> The epoch number at which the training was saved
    output: loss_save -> An array of all the saved batch losses during training
    output: optimizer -> The current state of the optimiser with its updated learning rates from training'''
    
    net = Net().cuda() # Initialize model  
    checkpoint = torch.load(filename) # load checkpoint data
    net.load_state_dict(checkpoint['state_dict'])
    epoch = checkpoint['epoch']
    losses = checkpoint['losses']
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    optimizer.load_state_dict(checkpoint['optimizer'])
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=PATIENCE, factor=SCHFACTOR, cooldown=CD, threshold=1e-2)
    # scheduler.best = checkpoint['best_loss']
    # scheduler.num_bad_epochs = checkpoint['bad_epoch']
    
    print("Loaded checkpoint: " + filename)
    return net, epoch, losses, optimizer#, scheduler

In [ ]:
def pokePlot(images, vae, ROW, COL):
    f, axarr = plt.subplots(ROW,COL, figsize=(15,ROW*4))
    for row in range(ROW//2):
        for col in range(COL):
            image = images[col+(COL*row),:,:,:].unsqueeze(0)
            axarr[2*row,col].imshow(image.squeeze().numpy())
            image = image.permute(0,3,1,2)
            x_out, z_mean, z_logvar = vae(Variable(image.float()).cuda())
            x_out = x_out.permute(0,2,3,1)
            axarr[2*row+1,col].imshow(x_out.data.cpu().squeeze().numpy())
    plt.show()

In [ ]:
# Training
label = "POKEVAE"
row = 4
col = 4

def train(model,optimizer,dataloader, epoch, max_epochs=5000):
    losses = []
    bces = []
    kls = []
    for _ in range(max_epochs):
        epoch += 1
        for images in dataloader:
            image_in = images.permute(0,3,1,2)
            x_in = Variable(image_in.float()).cuda()
            optimizer.zero_grad()
            x_out, z_mu, z_logvar = model(x_in)
            loss, bce, kl = criterion(x_out,x_in,z_mu,z_logvar)
            loss.backward()
            optimizer.step()
            losses.append(loss.data[0])
            bces.append(bce.data[0])
            kls.append(kl.data[0])
            
            if epoch % 10 == 0 and epoch != 0:
                save_file = label + "_epoch_{:06d}".format(epoch) + '.pth'
                torch.save({
                    'epoch': epoch,
                    'state_dict': net.state_dict(),
                    'optimizer' : optimizer.state_dict(),
                    'losses' : losses
                }, "checkpoints/" + save_file)
                
        clear_output(wait=True)
        pokePlot(images, net, row, col)
        print(epoch, 'Loss: {:3f}'.format(loss.data[0]))
        
    return losses, bces, kls
 
net, epoch, losses, optimizer = load_checkpoint("./checkpoints/POKEVAE_epoch_000680.pth")
train_losses, bces, kls = train(net,optimizer,pokeLoader, epoch)
plt.figure(figsize=(10,5))
plt.plot(train_losses)
plt.show()

plt.figure(figsize=(10,5))
plt.plot(bces)
plt.show()

plt.figure(figsize=(10,5))
plt.plot(kls)
plt.show()

In [ ]:
# dataiter = iter(pokeLoader)

# image = dataiter.next()

pokemon = mpimg.imread("./Pokemon/electrode.jpg")
pokemon = cv2.resize(pokemon, (RESIZE,RESIZE))
pokemon = (pokemon - np.min(pokemon))/np.max(pokemon - np.min(pokemon))
image = torch.from_numpy(pokemon).unsqueeze(0)

image = image[0,:,:,:].unsqueeze(0)
plt.imshow((image.squeeze().numpy()))
plt.show()
image = image.permute(0,3,1,2)
x_out, z_mean, z_logvar = net(Variable(image.float()).cuda())
x_out = x_out.permute(0,2,3,1)
plt.imshow(x_out.data.cpu().squeeze().numpy())
plt.show()
z = net.latent(z_mean, z_logvar)

In [ ]:
def sweep(image, dim, min_range, max_range, step):
#     plt.imshow((image.squeeze().numpy()))
#     plt.show()
    image = image.permute(0,3,1,2)
    z_mean, z_logvar = net.encoder(Variable(image.float()).cuda())
    z = net.latent(z_mean,z_logvar)
    for i, sw in enumerate(range(min_range, max_range, step)):
        z[0][dim] = sw/10
        x_out = net.decoder(z)
        x_out = x_out.permute(0,2,3,1)
        im = (np.floor(x_out.data.cpu().squeeze().numpy() * 255)).astype(np.uint8)
        plt.imsave("./sweep/" + str(dim) + "/" + "{0:03d}".format(i) + ".png", im)
    generate_animation("./sweep/" + str(dim) + "/", dim, (max_range - min_range)//step)

In [ ]:
def generate_animation(path, dim, num):
    images = []
    for e in range(num):
        img_name = path + "{0:03d}".format(e) + '.png'
        images.append(mpimg.imread(img_name))
    imageio.mimsave("./sweep/gif/" + 'dim' + str(dim) + '_generate_animation.gif', images, fps=30)


In [ ]:
image = image.permute(0,2,3,1)
for i in range(256):
    os.system("mkdir ./sweep/"+str(i))
    sweep(image, i, -300, 300, 5)

In [ ]:
# Visualize test data encodings on the latent space
def visualize_encoder(model,dataloader):
    z_means_x, z_means_y, all_labels = [], [], []
    
    for images in iter(dataloader):
        images = images.permute(0,3,1,2)
        z_means,_ = model.encoder(Variable(images.float()))
        z_means_x = np.append(z_means_x,z_means[:,0].data.numpy())
        z_means_y = np.append(z_means_y,z_means[:,1].data.numpy())
#         all_labels = np.append(all_labels,labels.numpy())
        
    plt.figure(figsize=(6.5,5))
    plt.scatter(z_means_x,z_means_y,cmap='inferno')
    plt.show()

visualize_encoder(net,pokeLoader)

In [ ]:
# Visualize moving average of losses
def visualize_losses_moving_average(losses,window=50,boundary='valid'):
    mav_losses = np.convolve(losses,np.ones(window)/window,boundary)
    corrected_mav_losses = np.append(np.full(window-1,np.nan),mav_losses)
    plt.figure(figsize=(10,5))
    plt.plot(losses)
    plt.plot(corrected_mav_losses)
    plt.ylim(ylim)
    plt.show()

visualize_losses_moving_average(train_losses)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, _ = data

        # wrap them in Variable
        inputs = Variable(inputs)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        x_out, z_mean, z_logvar = net(inputs)
        loss = criterion(x_out, inputs, z_mean, z_logvar)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

#### Testing Network

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))

#Predict with network
outputs, _, _ = net(Variable(images))

# _, predicted = torch.max(outputs.data, 1)

# print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
#                               for j in range(4)))


imshow(torchvision.utils.make_grid(outputs.data))

In [ ]:
#Test network accuracy

correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs, _, _ = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

#### RUN

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from scipy.stats import norm
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import MNIST
from torchvision.utils import make_grid as make_image_grid
from tqdm import tnrange

torch.manual_seed(2017) # reproducability
sns.set_style('dark')
%matplotlib inline

In [ ]:
# Model
class VAE(nn.Module):
    def __init__(self,latent_dim=20,hidden_dim=500):
        super(VAE,self).__init__()
        self.fc_e = nn.Linear(784,hidden_dim)
        self.fc_mean = nn.Linear(hidden_dim,latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim,latent_dim)
        self.fc_d1 = nn.Linear(latent_dim,hidden_dim)
        self.fc_d2 = nn.Linear(hidden_dim,784)
            
    def encoder(self,x_in):
        x = F.relu(self.fc_e(x_in.view(-1,784)))
        mean = self.fc_mean(x)
        logvar = self.fc_logvar(x)
        return mean, logvar
    
    def decoder(self,z):
        z = F.relu(self.fc_d1(z))
        x_out = F.sigmoid(self.fc_d2(z))
        return x_out.view(-1,1,28,28)
    
    def sample_normal(self,mean,logvar):
        # Using torch.normal(means,sds) returns a stochastic tensor which we cannot backpropogate through.
        # Instead we utilize the 'reparameterization trick'.
        # http://stats.stackexchange.com/a/205336
        # http://dpkingma.com/wordpress/wp-content/uploads/2015/12/talk_nips_workshop_2015.pdf
        sd = torch.exp(logvar*0.5)
        e = Variable(torch.randn(sd.size())) # Sample from standard normal
        z = e.mul(sd).add_(mean)
        return z
    
    def forward(self,x_in):
        z_mean, z_logvar = self.encoder(x_in)
        z = self.sample_normal(z_mean,z_logvar)
        x_out = self.decoder(z)
        return x_out, z_mean, z_logvar

model = VAE()

In [ ]:
# Loss function
def criterion(x_out,x_in,z_mu,z_logvar):
    bce_loss = F.binary_cross_entropy(x_out,x_in,size_average=False)
    kld_loss = -0.5 * torch.sum(1 + z_logvar - (z_mu ** 2) - torch.exp(z_logvar))
    loss = (bce_loss + kld_loss) / x_out.size(0) # normalize by batch size
    return loss

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# Data loaders
trainloader = DataLoader(
    MNIST(root='./MNIST',train=True,download=True,transform=transforms.ToTensor()),
    batch_size=128,shuffle=True)
testloader = DataLoader(
    MNIST(root='./MNIST',train=False,download=True,transform=transforms.ToTensor()),
    batch_size=128,shuffle=True)

In [ ]:
# Training
def train(model,optimizer,dataloader,epochs=15):
    losses = []
    for epoch in range(epochs):
        for images,_ in dataloader:
            x_in = Variable(images)
            optimizer.zero_grad()
            x_out, z_mu, z_logvar = model(x_in)
            loss = criterion(x_out,x_in,z_mu,z_logvar)
            loss.backward()
            optimizer.step()
            losses.append(loss.data[0])
        print('Loss: {:4f}'.format(loss.data[0]))
    return losses

train_losses = train(model,optimizer,trainloader)
plt.figure(figsize=(10,5))
plt.plot(train_losses)
plt.show()